In [1]:
import pandas as pd
import numpy as np

# --- Load ---
df = pd.read_csv("results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv")

# Keep only relevant cols
cols = ["simulation_run","timestamp","status","case_id","activity","resource","end_time","cycle_time","data","scenario","l","method"]
df = df[cols].copy()

# Parse numeric
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
df["end_time"] = pd.to_numeric(df["end_time"], errors="coerce")

# --- 1) basic schema sanity ---
required = ["timestamp","status","case_id","activity"]
missing_cols = [c for c in required if c not in df.columns]
assert not missing_cols, f"Missing columns: {missing_cols}"
assert df[required].isnull().sum().sum() == 0, "Nulls found in required fields"

# --- 2) reconstruct task instances (robust pairing) ---
# Keep only task lifecycle rows (exclude gateways and START/COMPLETE for case)
task_rows = df[df["activity"].ne("START") & df["status"].isin(["queued","running"])].copy()

# Sort so cumcounts respect time
task_rows = task_rows.sort_values(["case_id","activity","timestamp"]).copy()

# Split and make occurrence indices *per status* (not mixed)
queued = task_rows[task_rows["status"]=="queued"].copy()
queued["occ_q"] = queued.groupby(["case_id","activity"]).cumcount()

running = task_rows[task_rows["status"]=="running"].copy()
running["occ_r"] = running.groupby(["case_id","activity"]).cumcount()

# Merge kth queued with kth running for each (case, activity)
inst = pd.merge(
    queued[["case_id","activity","occ_q","timestamp"]].rename(columns={"timestamp":"queued_ts"}),
    running[["case_id","activity","occ_r","timestamp","end_time","resource"]].rename(
        columns={"timestamp":"start_ts","end_time":"end_ts","resource":"resource_name"}),
    left_on=["case_id","activity","occ_q"],
    right_on=["case_id","activity","occ_r"],
    how="inner",
).drop(columns=["occ_q","occ_r"])

print("num task instances:", len(inst))
print("sample inst:\n", inst.head())

# Durations
inst["wait"] = inst["start_ts"] - inst["queued_ts"]
inst["service"] = inst["end_ts"] - inst["start_ts"]
inst["sojourn"] = inst["end_ts"] - inst["queued_ts"]

# Sanity: no negatives
neg = inst[(inst["wait"]<0) | (inst["service"]<0) | (inst["sojourn"]<0)]
assert neg.empty, f"Negative times found:\n{neg.head()}"


# --- 3) FIFO discipline per station (warning-free) ---
tmp = inst.sort_values(["activity","queued_ts","start_ts"]).copy()
eps = 1e-9
tmp["dec"] = tmp.groupby("activity")["start_ts"].diff() < -eps

fifo_check = (
    tmp.groupby("activity", as_index=False)
       .agg(num_instances=("start_ts","size"),
            fifo_violations=("dec","sum"))
)
fifo_check["fifo_violations"] = fifo_check["fifo_violations"].astype(int)

# --- 4) routing/QC probabilities (position-based, handles same-timestamp + END) ---

# Sort by case then time, and give each row an order within the case
df_sorted = df.sort_values(["case_id", "timestamp"]).copy()
df_sorted["row_in_case"] = df_sorted.groupby("case_id").cumcount()

# Grab gateway rows (after they have row_in_case)
gates = df_sorted[df_sorted["status"] == "gateway"][["case_id","timestamp","activity","row_in_case"]].copy()
gates = gates.sort_values(["case_id","row_in_case"]).reset_index(drop=True)

def next_step_after_gateway(case_id, row_idx_after):
    """
    Return the 'next activity' chosen after a gateway for a given case, by scanning
    the subsequent rows (in order) for that case:
      - first 'queued' row -> its activity is the chosen branch
      - if none and gateway was QC_AFTER_PACKAGING, treat next COMPLETE as END
    """
    sub = df_sorted[(df_sorted["case_id"] == case_id) & (df_sorted["row_in_case"] > row_idx_after)]
    if sub.empty:
        return np.nan
    nxt_q = sub.loc[sub["status"] == "queued"]
    if not nxt_q.empty:
        return nxt_q.iloc[0]["activity"]
    # No queued afterwards; for packaging gateway the case may complete next
    nxt_c = sub.loc[sub["status"] == "COMPLETE"]
    if not nxt_c.empty:
        return "END"
    return np.nan

gates["next_activity"] = [
    next_step_after_gateway(cid, r)
    for cid, r in gates[["case_id","row_in_case"]].itertuples(index=False, name=None)
]

# Empirical routing per gateway label
routing_emp = (
    gates.groupby("activity")["next_activity"]
         .value_counts(normalize=True, dropna=True)
         .rename("prob")
         .reset_index()
)

# Expected (from your config)
expected = {
    "QC_AFTER_MOULDING":  {"MOULDING": 0.05, "ASSEMBLY_1": 0.95},
    "QC_AFTER_ASSEMBLY1": {"ASSEMBLY_1": 0.05, "ASSEMBLY_2": 0.95},
    "QC_AFTER_ASSEMBLY2": {"ASSEMBLY_2": 0.05, "SORTING":    0.95},
    "QC_AFTER_SORTING":   {"SORTING":   0.05, "PACKAGING":   0.95},
    "QC_AFTER_PACKAGING": {"PACKAGING": 0.05, "END":         0.95},
}
routing_exp = pd.DataFrame(
    [{"activity": g, "next_activity": nxt, "expected": p}
     for g, dd in expected.items() for nxt, p in dd.items()]
)

routing = (routing_exp
           .merge(routing_emp, on=["activity","next_activity"], how="left")
           .fillna({"prob": 0.0}))
routing["diff"] = routing["prob"] - routing["expected"]

print("\nRouting empirical vs expected (position-based):")
print(routing.sort_values(["activity","next_activity"]))

# How many gateways had 0-wait to the next station?
merge_for_wait = gates.merge(
    inst[["case_id","activity","queued_ts","start_ts"]],
    left_on=["case_id","next_activity"], right_on=["case_id","activity"],
    how="left", suffixes=("_gate","_next")
)
zero_wait_share = (merge_for_wait["start_ts"] - merge_for_wait["timestamp"] <= 1e-9).mean()
print("Share of gateways with immediate (zero-wait) handoff:", zero_wait_share)


# --- 5) service-time by resource ---
svc = inst.dropna(subset=["resource_name","service"]).copy()
svc_stats = svc.groupby("resource_name")["service"].agg(["count","mean","std","min","max"]).reset_index()

# Rough exponential check: for Exp, mean ≈ std; ratio near 1 is indicative (not proof)
svc_stats["std_over_mean"] = svc_stats["std"]/svc_stats["mean"]

# --- 6) utilization & throughput ---
T = df["timestamp"].max()  # sim horizon (approx)
busy = (
    svc.groupby("resource_name", as_index=False)["service"]
       .sum()
       .rename(columns={"service":"busy_time"})
)
busy["utilization"] = busy["busy_time"] / T

# Station-level KPIs
station = inst.groupby("activity").agg(
    arrivals=("activity","size"),
    mean_wait=("wait","mean"),
    mean_service=("service","mean"),
    mean_sojourn=("sojourn","mean")
).reset_index()

# --- 7) Little's Law check: L ≈ λ W at station level ---
# λ = arrivals / T; W = mean_sojourn at station; L_hat = λ*W
station["lambda"] = station["arrivals"] / T
station["L_hat"] = station["lambda"] * station["mean_sojourn"]

# (Optional, time-weighted L from queue-length trace would be more exact.)

# --- Summaries to print or save ---
print("\nFIFO violations per station (should be 0):")
print(fifo_check.sort_values("fifo_violations", ascending=False).head(10))

print("\nRouting empirical vs expected (diff near 0):")
print(routing.sort_values(["activity","next_activity"]))

print("\nService by resource (std/mean ~ 1 for Exp):")
print(svc_stats.sort_values("std_over_mean"))

print("\nResource utilization (sanity):")
print(busy.sort_values("utilization", ascending=False).head(10))

print("\nStation KPIs + Little's Law proxy:")
print(station.sort_values("L_hat", ascending=False).head(10))

# --- Stability audit: capacity vs offered load, with warm-up trim ---

warmup = df["timestamp"].max() * 0.05  # trim first 5% to reduce transient bias
inst_ss = inst[inst["queued_ts"] >= warmup].copy()

# Per-station arrival rate λ_station (jobs/min)
T_eff = df["timestamp"].max() - warmup
arrivals = inst_ss.groupby("activity")["activity"].size().rename("arrivals").reset_index()
arrivals["lambda_station"] = arrivals["arrivals"] / T_eff

# Mean service time per (activity, resource) then aggregate to station capacity
svc_ss = inst_ss.dropna(subset=["resource_name","service"]).copy()
rs = (svc_ss.groupby(["activity","resource_name"])["service"]
            .mean()
            .rename("mean_service")
            .reset_index())

# Station capacity (jobs/min) = sum over resources of 1 / E[S_resource]
cap = (rs.assign(mu=lambda d: 1.0 / d["mean_service"])
         .groupby("activity")["mu"].sum()
         .rename("capacity")
         .reset_index())

# Merge and compute utilization proxy and safety margin
stab = (arrivals.merge(cap, on="activity", how="left"))
stab["rho_hat"] = stab["lambda_station"] / stab["capacity"]
stab["headroom"] = 1.0 - stab["rho_hat"]

print("\nStability headroom by station (ρ_hat < 1 is required; aim for headroom ≥ 0.05):")
print(stab.sort_values("rho_hat", ascending=False))

import numpy as np

def queue_trace(inst_df, act):
    g = inst_df[inst_df["activity"]==act][["queued_ts","start_ts"]].copy()
    if g.empty: 
        return None
    ups   = g.groupby("queued_ts").size().rename("delta")
    downs = (g.groupby("start_ts").size() * -1).rename("delta")
    deltas = pd.concat([ups, downs], axis=0).groupby(level=0).sum().sort_index()
    tr = deltas.cumsum().reset_index().rename(columns={"index":"time","delta":"qlen"})
    tr["qlen"] = tr["qlen"].clip(lower=0)
    return tr

def end_trend_slope(trace, tail_frac=0.2):
    if trace is None or len(trace) < 5: 
        return np.nan
    n0 = int(len(trace)*(1-tail_frac))
    tail = trace.iloc[max(n0,0):].copy()
    if len(tail) < 3: 
        return np.nan
    # simple least-squares slope of qlen on time
    x = tail["time"].values
    y = tail["qlen"].values
    A = np.vstack([x, np.ones_like(x)]).T
    slope, _ = np.linalg.lstsq(A, y, rcond=None)[0]
    return slope  # jobs per minute near the end

acts = sorted(inst_ss["activity"].unique())
trend = []
for act in acts:
    tr = queue_trace(inst_ss, act)
    slope = end_trend_slope(tr, tail_frac=0.3)  # check last 30% of events
    trend.append({"activity": act, "end_slope_q_per_min": slope})

trend_df = pd.DataFrame(trend).sort_values("end_slope_q_per_min", ascending=False)
print("\nQueue end-trend slope (jobs/min; ≈0 is good; persistently >0 indicates growing backlog):")
print(trend_df)



num task instances: 72378
sample inst:
    case_id    activity  queued_ts  start_ts     end_ts     resource_name
0        0  ASSEMBLY_1   0.264408  0.264408  19.859716  LINE_1_ASSEMBLY1
1        0  ASSEMBLY_1   3.680903  3.680903   5.773112  LINE_1_ASSEMBLY1
2        0  ASSEMBLY_1   4.878237  4.878237   5.628757  LINE_1_ASSEMBLY1
3        0  ASSEMBLY_1   6.279048  6.279048  15.493677  LINE_1_ASSEMBLY1
4        0  ASSEMBLY_1   6.416882  6.416882   8.926294  LINE_1_ASSEMBLY1


KeyboardInterrupt: 

In [18]:
# ============================ CONFIG ============================
GLOB_PATTERN   = "results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv"
WARMUP_FRAC    = 0.10   # 10% warm-up for steady-state metrics
COOLDOWN_FRAC  = 0.02   # 2% tail trim
EPS            = 1e-9
SAVE_DISCOVERY = False  # set True to save case-trimmed discovery log
DISCOVERY_OUT  = "results/discovery_steady_state.csv"
# ===============================================================

import pandas as pd, numpy as np, glob, os

# ---------- Load ----------
files = glob.glob(GLOB_PATTERN)
if not files:
    raise FileNotFoundError(f"No files matched: {GLOB_PATTERN}")

df = pd.concat([pd.read_csv(p) for p in files], ignore_index=True)
# types
df["timestamp"]      = pd.to_numeric(df["timestamp"], errors="coerce")
df["end_time"]       = pd.to_numeric(df.get("end_time"), errors="coerce")
df["simulation_run"] = pd.to_numeric(df["simulation_run"], errors="coerce").fillna(0).astype(int)

# unique case key across runs
df["global_case_id"] = df["simulation_run"].astype(str) + "-" + df["case_id"].astype(str)

# ---------- Helpers ----------
expected = {
    "QC_AFTER_MOULDING":  {"MOULDING": 0.05, "ASSEMBLY_1": 0.95},
    "QC_AFTER_ASSEMBLY1": {"ASSEMBLY_1": 0.05, "ASSEMBLY_2": 0.95},
    "QC_AFTER_ASSEMBLY2": {"ASSEMBLY_2": 0.05, "SORTING":    0.95},
    "QC_AFTER_SORTING":   {"SORTING":   0.05, "PACKAGING":   0.95},
    "QC_AFTER_PACKAGING": {"PACKAGING": 0.05, "END":         0.95},
}
routing_exp = pd.DataFrame(
    [{"activity": g, "next_activity": nxt, "expected": p}
     for g, dd in expected.items() for nxt, p in dd.items()]
)

def pair_instances(df_run):
    """Robust queued↔running pairing per (case, activity, occurrence)."""
    tr = df_run[(df_run["activity"].ne("START")) &
                (df_run["status"].isin(["queued","running"]))].copy()
    tr = tr.sort_values(["case_id","activity","timestamp"])

    q = tr[tr["status"]=="queued"].copy()
    q["occ_q"] = q.groupby(["case_id","activity"]).cumcount()

    r = tr[tr["status"]=="running"].copy()
    r["occ_r"] = r.groupby(["case_id","activity"]).cumcount()

    inst = pd.merge(
        q[["case_id","activity","occ_q","timestamp"]].rename(columns={"timestamp":"queued_ts"}),
        r[["case_id","activity","occ_r","timestamp","end_time","resource"]].rename(
            columns={"timestamp":"start_ts","end_time":"end_ts","resource":"resource_name"}),
        left_on=["case_id","activity","occ_q"],
        right_on=["case_id","activity","occ_r"],
        how="inner",
    ).drop(columns=["occ_q","occ_r"])

    inst["wait"]    = inst["start_ts"] - inst["queued_ts"]
    inst["service"] = inst["end_ts"]   - inst["start_ts"]
    inst["sojourn"] = inst["end_ts"]   - inst["queued_ts"]
    return inst

def fifo_violations(inst):
    """Check monotone nondecreasing start_ts after sorting by queued_ts."""
    if inst.empty:
        return pd.DataFrame({"activity": [], "fifo_violations": []})
    tmp = inst.sort_values(["activity","queued_ts","start_ts"]).copy()
    tmp["dec"] = tmp.groupby("activity")["start_ts"].diff() < -EPS
    return (tmp.groupby("activity")["dec"].sum()
               .astype(int).rename("fifo_violations").reset_index())

def routing_empirical(df_run):
    """Position-based extractor with END handling."""
    s = df_run.sort_values(["case_id","timestamp"]).copy()
    s["row_in_case"] = s.groupby("case_id").cumcount()
    gates = s[s["status"]=="gateway"][["case_id","timestamp","activity","row_in_case"]].copy()

    def next_step(case_id, row_after):
        sub = s[(s["case_id"]==case_id) & (s["row_in_case"]>row_after)]
        if sub.empty: return np.nan
        nxtq = sub.loc[sub["status"]=="queued"]
        if not nxtq.empty: return nxtq.iloc[0]["activity"]
        nxtc = sub.loc[sub["status"]=="COMPLETE"]
        if not nxtc.empty: return "END"
        return np.nan

    if gates.empty:
        out = routing_exp.copy()
        out["prob"] = np.nan
        out["diff"] = np.nan
        return out

    gates["next_activity"] = [next_step(cid, r)
                              for cid, r in gates[["case_id","row_in_case"]].itertuples(index=False, name=None)]
    emp = (gates.groupby("activity")["next_activity"]
                 .value_counts(normalize=True, dropna=True)
                 .rename("prob").reset_index())
    rout = (routing_exp.merge(emp, on=["activity","next_activity"], how="left")
                      .fillna({"prob":0.0}))
    rout["diff"] = rout["prob"] - rout["expected"]
    return rout

def capacity_headroom(inst_ss, df_run_ss):
    """λ_station vs capacity (sum of per-resource μ̂)."""
    if inst_ss.empty or df_run_ss.empty:
        return pd.DataFrame(columns=["activity","arrivals","lambda_station","capacity","rho_hat","headroom"])
    # arrivals per station in steady-state window
    T_eff = df_run_ss["timestamp"].max() - df_run_ss["timestamp"].min()
    arr = (inst_ss.groupby("activity")["activity"].size()
           .rename("arrivals").reset_index())
    arr["lambda_station"] = arr["arrivals"] / max(T_eff, EPS)

    # capacity via per-resource μ̂ = n / busy_time
    svc = inst_ss.dropna(subset=["resource_name","service"]).copy()
    rt = (svc.groupby(["activity","resource_name"])["service"]
              .agg(n="size", busy="sum").reset_index())
    rt["mu_hat"] = rt["n"] / rt["busy"]
    cap = (rt.groupby("activity")["mu_hat"].sum()
             .rename("capacity").reset_index())

    stab = arr.merge(cap, on="activity", how="left")
    stab["rho_hat"] = stab["lambda_station"] / stab["capacity"]
    stab["headroom"] = 1 - stab["rho_hat"]
    return stab

def queue_end_trend(inst_ss, tail_frac=0.3):
    """Least-squares slope of qlen over time on last fraction; ~0 means stable."""
    def qtrace(act):
        g = inst_ss[inst_ss["activity"]==act][["queued_ts","start_ts"]].copy()
        if g.empty: return None
        ups   = g.groupby("queued_ts").size().rename("d")
        downs = (g.groupby("start_ts").size()*-1).rename("d")
        deltas = pd.concat([ups, downs], axis=0).groupby(level=0).sum().sort_index()
        tr = deltas.cumsum().reset_index().rename(columns={"index":"time", "d":"qlen"})
        tr["qlen"] = tr["qlen"].clip(lower=0)
        return tr

    rows=[]
    for act in sorted(inst_ss["activity"].unique()):
        tr = qtrace(act)
        if tr is None or len(tr)<5:
            rows.append({"activity":act, "end_slope_q_per_min": np.nan})
            continue
        n0 = int(len(tr)*(1-tail_frac))
        tail = tr.iloc[max(n0,0):]
        x = tail["time"].values; y = tail["qlen"].values
        A = np.vstack([x, np.ones_like(x)]).T
        slope, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        rows.append({"activity":act, "end_slope_q_per_min": slope})
    return pd.DataFrame(rows)

# ---------- Per-run audits ----------
per_run = []

for run, df_run in df.groupby("simulation_run", sort=True):
    t_max = df_run["timestamp"].max()
    t0 = t_max*WARMUP_FRAC
    t1 = t_max*(1-COOLDOWN_FRAC)

    # steady-state event window for metrics
    df_run_ss_events = df_run[(df_run["timestamp"]>=t0) & (df_run["timestamp"]<=t1)].copy()

    # robust pairing on full run; then trim by queued_ts for SS
    inst_full = pair_instances(df_run)
    inst_ss   = inst_full[inst_full["queued_ts"]>=t0].copy()

    # audits
    fifo   = fifo_violations(inst_ss);     fifo["simulation_run"] = run
    rout   = routing_empirical(df_run_ss_events); rout["simulation_run"] = run
    stab   = capacity_headroom(inst_ss, df_run_ss_events); stab["simulation_run"] = run
    slope  = queue_end_trend(inst_ss, tail_frac=0.3); slope["simulation_run"] = run

    # waits (quantiles) per station
    waits = (inst_ss.groupby("activity")["wait"]
             .quantile([0.5,0.9,0.95]).unstack().rename(columns={0.5:"p50",0.9:"p90",0.95:"p95"})
             .reset_index())
    waits["simulation_run"] = run

    per_run.append((fifo, rout, stab, slope, waits))

fifo_all   = pd.concat([x[0] for x in per_run], ignore_index=True)
routing_all= pd.concat([x[1] for x in per_run], ignore_index=True)
stab_all   = pd.concat([x[2] for x in per_run], ignore_index=True)
slope_all  = pd.concat([x[3] for x in per_run], ignore_index=True)
waits_all  = pd.concat([x[4] for x in per_run], ignore_index=True)

# ---------- Print summaries ----------
print("\n=== FIFO violations per run (should be all zeros) ===")
if fifo_all.empty:
    print("No task instances in steady-state window.")
else:
    print(fifo_all.pivot_table(index="simulation_run", columns="activity", values="fifo_violations", fill_value=0))

print("\n=== Routing diff from expected (mean ± std across runs) ===")
routing_sum = (routing_all
               .groupby(["activity","next_activity"])
               .agg(diff_mean=("diff","mean"), diff_std=("diff","std"))
               .reset_index()
               .sort_values(["activity","next_activity"]))
print(routing_sum)

print("\n=== Stability headroom: bottleneck per run and min headroom ===")
bot = (stab_all.sort_values(["simulation_run","rho_hat"], ascending=[True,False])
               .groupby("simulation_run").head(1)
               .rename(columns={"activity":"bottleneck"}))
print(bot[["simulation_run","bottleneck","rho_hat","headroom"]])
if not bot.empty:
    print("Min headroom across runs:", bot["headroom"].min())

print("\n=== Queue end-trend slopes (mean across runs; ≈0 is good) ===")
print(slope_all.groupby("activity")["end_slope_q_per_min"].mean().sort_values(ascending=False))

print("\n=== Wait quantiles (p50/p90/p95) averaged across runs ===")
print(waits_all.groupby("activity")[["p50","p90","p95"]].mean().round(3))

# ---------- Extra consistency checks (starts==completes, cycle time) ----------
print("\n=== Starts vs Completes (case-trimmed) ===")
# case-level metadata
case_start = (df[df["status"]=="START"]
              .groupby(["simulation_run","case_id"])["timestamp"].min()
              .rename("case_start"))
case_last  = df.groupby(["simulation_run","case_id"])["timestamp"].max().rename("case_last")
case_end   = (df[df["status"]=="COMPLETE"]
              .groupby(["simulation_run","case_id"])["timestamp"].max()
              .rename("case_end"))
case_meta  = (pd.concat([case_start, case_end, case_last], axis=1)
                .reset_index())
case_meta["case_end_fallback"] = case_meta["case_end"].fillna(case_meta["case_last"])

# keep cases fully inside SS window for discovery & consistency
global_tmax = df["timestamp"].max()
warmup_cut  = global_tmax * WARMUP_FRAC
cool_cut    = global_tmax * (1-COOLDOWN_FRAC)
keep_cases = case_meta[
    (case_meta["case_start"] >= warmup_cut) &
    (case_meta["case_end_fallback"] <= cool_cut)
][["simulation_run","case_id"]]
df_ss_cases = df.merge(keep_cases.assign(_keep=1), on=["simulation_run","case_id"], how="inner")

starts = (df_ss_cases[df_ss_cases["status"]=="START"]
          .groupby("simulation_run")["case_id"].nunique())
completes = (df_ss_cases[df_ss_cases["status"]=="COMPLETE"]
             .groupby("simulation_run")["case_id"].nunique())
starts_vs_completes = pd.concat([starts.rename("n_start"), completes.rename("n_complete")], axis=1).fillna(0).astype(int)
starts_vs_completes["match"] = starts_vs_completes["n_start"] == starts_vs_completes["n_complete"]
print(starts_vs_completes)
print("All runs balanced:", starts_vs_completes["match"].all())

# cycle time consistency on COMPLETE rows
comp = df_ss_cases[df_ss_cases["status"]=="COMPLETE"].copy()
comp = comp.merge(case_start.rename("case_start_time").reset_index(),
                  on=["simulation_run","case_id"], how="left")
comp["ct_recalc"] = comp["timestamp"] - comp["case_start_time"]
if "cycle_time" in comp.columns:
    comp["cycle_time"] = pd.to_numeric(comp["cycle_time"], errors="coerce")
    comp["ct_diff"] = (comp["ct_recalc"] - comp["cycle_time"]).abs()
    print("Cycle-time max abs diff:", (comp["ct_diff"].max() if not comp.empty else 0.0))
else:
    print("No 'cycle_time' column found; skipped direct comparison.")

# ---------- Optional: save discovery-ready log ----------
if SAVE_DISCOVERY:
    os.makedirs(os.path.dirname(DISCOVERY_OUT), exist_ok=True)
    df_ss_cases.to_csv(DISCOVERY_OUT, index=False)
    print(f"\nSaved discovery dataset: {DISCOVERY_OUT}  "
          f"(cases: {df_ss_cases['case_id'].nunique()}, events: {len(df_ss_cases)})")



=== FIFO violations per run (should be all zeros) ===
activity        ASSEMBLY_1  ASSEMBLY_2  MOULDING  PACKAGING  SORTING
simulation_run                                                      
0                      0.0         0.0       0.0        0.0      0.0
1                      0.0         0.0       0.0        0.0      0.0
2                      0.0         0.0       0.0        0.0      0.0
3                      0.0         0.0       0.0        0.0      0.0
4                      0.0         0.0       0.0        0.0      0.0
5                      0.0         0.0       0.0        0.0      0.0
6                      0.0         0.0       0.0        0.0      0.0
7                      0.0         0.0       0.0        0.0      0.0
8                      0.0         0.0       0.0        0.0      0.0
9                      0.0         0.0       0.0        0.0      0.0

=== Routing diff from expected (mean ± std across runs) ===
             activity next_activity  diff_mean  diff_std

In [19]:
import pandas as pd
import numpy as np

# --- Load ---
df = pd.read_csv("results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv")

# Keep only relevant cols
cols = ["simulation_run","timestamp","status","case_id","activity","resource","end_time","cycle_time","data","scenario","l","method"]
df = df[cols].copy()

# Parse numeric
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
df["end_time"] = pd.to_numeric(df["end_time"], errors="coerce")

# === CRITICAL: Create composite case identifier ===
df["run_case"] = df["simulation_run"].astype(str) + "_" + df["case_id"].astype(str)

# --- 1) basic schema sanity ---
required = ["timestamp","status","case_id","activity"]
missing_cols = [c for c in required if c not in df.columns]
assert not missing_cols, f"Missing columns: {missing_cols}"
assert df[required].isnull().sum().sum() == 0, "Nulls found in required fields"

# --- 2) reconstruct task instances (robust pairing) ---
# Keep only task lifecycle rows (exclude gateways and START/COMPLETE for case)
task_rows = df[df["activity"].ne("START") & df["status"].isin(["queued","running"])].copy()

# Sort so cumcounts respect time
task_rows = task_rows.sort_values(["run_case","activity","timestamp"]).copy()

# Split and make occurrence indices *per status* (not mixed)
queued = task_rows[task_rows["status"]=="queued"].copy()
queued["occ_q"] = queued.groupby(["run_case","activity"]).cumcount()

running = task_rows[task_rows["status"]=="running"].copy()
running["occ_r"] = running.groupby(["run_case","activity"]).cumcount()

# Merge kth queued with kth running for each (run_case, activity)
inst = pd.merge(
    queued[["simulation_run","run_case","activity","occ_q","timestamp"]].rename(columns={"timestamp":"queued_ts"}),
    running[["run_case","activity","occ_r","timestamp","end_time","resource"]].rename(
        columns={"timestamp":"start_ts","end_time":"end_ts","resource":"resource_name"}),
    left_on=["run_case","activity","occ_q"],
    right_on=["run_case","activity","occ_r"],
    how="inner",
).drop(columns=["occ_q","occ_r"])

print("num task instances:", len(inst))
print("sample inst:\n", inst.head())

# Durations
inst["wait"] = inst["start_ts"] - inst["queued_ts"]
inst["service"] = inst["end_ts"] - inst["start_ts"]
inst["sojourn"] = inst["end_ts"] - inst["queued_ts"]

# Sanity: no negatives
neg = inst[(inst["wait"]<0) | (inst["service"]<0) | (inst["sojourn"]<0)]
assert neg.empty, f"Negative times found:\n{neg.head()}"


# --- 3) FIFO discipline per station (warning-free) ---
# Check FIFO within each run separately
tmp = inst.sort_values(["simulation_run","activity","queued_ts","start_ts"]).copy()
eps = 1e-9
tmp["dec"] = tmp.groupby(["simulation_run","activity"])["start_ts"].diff() < -eps

fifo_check = (
    tmp.groupby(["simulation_run","activity"], as_index=False)
       .agg(num_instances=("start_ts","size"),
            fifo_violations=("dec","sum"))
)
fifo_check["fifo_violations"] = fifo_check["fifo_violations"].astype(int)

# Aggregate across runs
fifo_summary = (
    fifo_check.groupby("activity", as_index=False)
    .agg(
        total_instances=("num_instances","sum"),
        total_violations=("fifo_violations","sum"),
        runs_with_violations=("fifo_violations", lambda x: (x > 0).sum())
    )
)

# --- 4) routing/QC probabilities (position-based, handles same-timestamp + END) ---

# Sort by run_case then time, and give each row an order within the run_case
df_sorted = df.sort_values(["run_case", "timestamp"]).copy()
df_sorted["row_in_case"] = df_sorted.groupby("run_case").cumcount()

# Grab gateway rows (after they have row_in_case)
gates = df_sorted[df_sorted["status"] == "gateway"][["simulation_run","run_case","timestamp","activity","row_in_case"]].copy()
gates = gates.sort_values(["run_case","row_in_case"]).reset_index(drop=True)

def next_step_after_gateway(run_case, row_idx_after):
    """
    Return the 'next activity' chosen after a gateway for a given run_case, by scanning
    the subsequent rows (in order) for that case:
      - first 'queued' row -> its activity is the chosen branch
      - if none and gateway was QC_AFTER_PACKAGING, treat next COMPLETE as END
    """
    sub = df_sorted[(df_sorted["run_case"] == run_case) & (df_sorted["row_in_case"] > row_idx_after)]
    if sub.empty:
        return np.nan
    nxt_q = sub.loc[sub["status"] == "queued"]
    if not nxt_q.empty:
        return nxt_q.iloc[0]["activity"]
    # No queued afterwards; for packaging gateway the case may complete next
    nxt_c = sub.loc[sub["status"] == "COMPLETE"]
    if not nxt_c.empty:
        return "END"
    return np.nan

gates["next_activity"] = [
    next_step_after_gateway(rc, r)
    for rc, r in gates[["run_case","row_in_case"]].itertuples(index=False, name=None)
]

# Empirical routing per gateway label (aggregated across all runs)
routing_emp = (
    gates.groupby("activity")["next_activity"]
         .value_counts(normalize=True, dropna=True)
         .rename("prob")
         .reset_index()
)

# Expected (from your config)
expected = {
    "QC_AFTER_MOULDING":  {"MOULDING": 0.05, "ASSEMBLY_1": 0.95},
    "QC_AFTER_ASSEMBLY1": {"ASSEMBLY_1": 0.05, "ASSEMBLY_2": 0.95},
    "QC_AFTER_ASSEMBLY2": {"ASSEMBLY_2": 0.05, "SORTING":    0.95},
    "QC_AFTER_SORTING":   {"SORTING":   0.05, "PACKAGING":   0.95},
    "QC_AFTER_PACKAGING": {"PACKAGING": 0.05, "END":         0.95},
}
routing_exp = pd.DataFrame(
    [{"activity": g, "next_activity": nxt, "expected": p}
     for g, dd in expected.items() for nxt, p in dd.items()]
)

routing = (routing_exp
           .merge(routing_emp, on=["activity","next_activity"], how="left")
           .fillna({"prob": 0.0}))
routing["diff"] = routing["prob"] - routing["expected"]

print("\nRouting empirical vs expected (aggregated across all runs):")
print(routing.sort_values(["activity","next_activity"]))

# How many gateways had 0-wait to the next station?
merge_for_wait = gates.merge(
    inst[["run_case","activity","queued_ts","start_ts"]],
    left_on=["run_case","next_activity"], right_on=["run_case","activity"],
    how="left", suffixes=("_gate","_next")
)
zero_wait_share = (merge_for_wait["start_ts"] - merge_for_wait["timestamp"] <= 1e-9).mean()
print("Share of gateways with immediate (zero-wait) handoff:", zero_wait_share)


# --- 5) service-time by resource (aggregated across runs) ---
svc = inst.dropna(subset=["resource_name","service"]).copy()
svc_stats = svc.groupby("resource_name")["service"].agg(["count","mean","std","min","max"]).reset_index()

# Rough exponential check: for Exp, mean ≈ std; ratio near 1 is indicative (not proof)
svc_stats["std_over_mean"] = svc_stats["std"]/svc_stats["mean"]

# --- 6) utilization & throughput (per run, then averaged) ---
# Calculate per run first
util_per_run = []
for run_id in df["simulation_run"].unique():
    df_run = df[df["simulation_run"] == run_id]
    inst_run = inst[inst["simulation_run"] == run_id]
    
    T = df_run["timestamp"].max()
    busy = (
        inst_run.groupby("resource_name", as_index=False)["service"]
           .sum()
           .rename(columns={"service":"busy_time"})
    )
    busy["utilization"] = busy["busy_time"] / T
    busy["simulation_run"] = run_id
    util_per_run.append(busy)

# Average utilization across runs
util_all = pd.concat(util_per_run, ignore_index=True)
util_summary = (
    util_all.groupby("resource_name")["utilization"]
    .agg(["mean","std","min","max"])
    .reset_index()
)

# Station-level KPIs (aggregated across runs)
station = inst.groupby("activity").agg(
    arrivals=("activity","size"),
    mean_wait=("wait","mean"),
    mean_service=("service","mean"),
    mean_sojourn=("sojourn","mean")
).reset_index()

# --- 7) Little's Law check: aggregate across runs ---
# Use total simulation time across all runs
total_sim_time = df.groupby("simulation_run")["timestamp"].max().sum()
station["lambda"] = station["arrivals"] / total_sim_time
station["L_hat"] = station["lambda"] * station["mean_sojourn"]

# --- Summaries to print or save ---
print("\nFIFO violations summary (should be 0):")
print(fifo_summary.sort_values("total_violations", ascending=False))

print("\nService by resource (std/mean ~ 1 for Exp):")
print(svc_stats.sort_values("std_over_mean"))

print("\nResource utilization (mean across runs):")
print(util_summary.sort_values("mean", ascending=False).head(10))

print("\nStation KPIs + Little's Law proxy:")
print(station.sort_values("L_hat", ascending=False).head(10))

# --- Stability audit: capacity vs offered load, with warm-up trim ---
# Do this per run, then average
stab_per_run = []

for run_id in df["simulation_run"].unique():
    df_run = df[df["simulation_run"] == run_id]
    inst_run = inst[inst["simulation_run"] == run_id]
    
    warmup = df_run["timestamp"].max() * 0.05
    inst_ss = inst_run[inst_run["queued_ts"] >= warmup].copy()
    
    T_eff = df_run["timestamp"].max() - warmup
    arrivals_run = inst_ss.groupby("activity")["activity"].size().rename("arrivals").reset_index()
    arrivals_run["lambda_station"] = arrivals_run["arrivals"] / T_eff
    
    svc_ss = inst_ss.dropna(subset=["resource_name","service"]).copy()
    rs = (svc_ss.groupby(["activity","resource_name"])["service"]
                .mean()
                .rename("mean_service")
                .reset_index())
    
    cap = (rs.assign(mu=lambda d: 1.0 / d["mean_service"])
             .groupby("activity")["mu"].sum()
             .rename("capacity")
             .reset_index())
    
    stab_run = arrivals_run.merge(cap, on="activity", how="left")
    stab_run["rho_hat"] = stab_run["lambda_station"] / stab_run["capacity"]
    stab_run["headroom"] = 1.0 - stab_run["rho_hat"]
    stab_run["simulation_run"] = run_id
    
    stab_per_run.append(stab_run)

# Average stability metrics across runs
stab_all = pd.concat(stab_per_run, ignore_index=True)
stab_summary = (
    stab_all.groupby("activity")
    .agg(
        mean_rho=("rho_hat","mean"),
        std_rho=("rho_hat","std"),
        mean_headroom=("headroom","mean"),
        min_headroom=("headroom","min")
    )
    .reset_index()
)

print("\nStability headroom by station (mean across runs; ρ < 1 required; aim for headroom ≥ 0.05):")
print(stab_summary.sort_values("mean_rho", ascending=False))

# --- Queue end-trend analysis (per run, then summarize) ---
def queue_trace(inst_df, act):
    g = inst_df[inst_df["activity"]==act][["queued_ts","start_ts"]].copy()
    if g.empty: 
        return None
    ups   = g.groupby("queued_ts").size().rename("delta")
    downs = (g.groupby("start_ts").size() * -1).rename("delta")
    deltas = pd.concat([ups, downs], axis=0).groupby(level=0).sum().sort_index()
    tr = deltas.cumsum().reset_index().rename(columns={"index":"time","delta":"qlen"})
    tr["qlen"] = tr["qlen"].clip(lower=0)
    return tr

def end_trend_slope(trace, tail_frac=0.2):
    if trace is None or len(trace) < 5: 
        return np.nan
    n0 = int(len(trace)*(1-tail_frac))
    tail = trace.iloc[max(n0,0):].copy()
    if len(tail) < 3: 
        return np.nan
    x = tail["time"].values
    y = tail["qlen"].values
    A = np.vstack([x, np.ones_like(x)]).T
    slope, _ = np.linalg.lstsq(A, y, rcond=None)[0]
    return slope

trend_per_run = []
for run_id in df["simulation_run"].unique():
    inst_run = inst[inst["simulation_run"] == run_id]
    warmup = df[df["simulation_run"] == run_id]["timestamp"].max() * 0.05
    inst_ss = inst_run[inst_run["queued_ts"] >= warmup]
    
    acts = sorted(inst_ss["activity"].unique())
    for act in acts:
        tr = queue_trace(inst_ss, act)
        slope = end_trend_slope(tr, tail_frac=0.3)
        trend_per_run.append({
            "simulation_run": run_id,
            "activity": act, 
            "end_slope_q_per_min": slope
        })

trend_df = pd.DataFrame(trend_per_run)
trend_summary = (
    trend_df.groupby("activity")["end_slope_q_per_min"]
    .agg(["mean","std","min","max"])
    .reset_index()
    .sort_values("mean", ascending=False)
)

print("\nQueue end-trend slope (mean across runs; ≈0 is good; persistently >0 indicates growing backlog):")
print(trend_summary)

num task instances: 72378
sample inst:
    simulation_run run_case    activity  queued_ts   start_ts     end_ts  \
0               0      0_0  ASSEMBLY_1  20.393651  20.393651  24.866156   
1               0      0_0  ASSEMBLY_2  24.866156  24.866156  26.676780   
2               0      0_0    MOULDING  18.537263  18.537263  20.393651   
3               0      0_0   PACKAGING  29.063644  29.063644  36.031979   
4               0      0_0     SORTING  26.676780  26.676780  29.063644   

        resource_name  
0    LINE_1_ASSEMBLY1  
1    ASSEMBLY2_LINE_1  
2  MOULDING_MACHINE_1  
3    PACKAGING_LINE_1  
4       SORTING_ROBOT  


KeyboardInterrupt: 

In [20]:
import pandas as pd
import numpy as np

# --- Load ---
df = pd.read_csv("results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv")

# Keep only relevant cols
cols = ["simulation_run","timestamp","status","case_id","activity","resource","end_time","cycle_time","data","scenario","l","method"]
df = df[cols].copy()

# Parse numeric
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
df["end_time"] = pd.to_numeric(df["end_time"], errors="coerce")

# --- 1) basic schema sanity ---
required = ["simulation_run","timestamp","status","case_id","activity"]
missing_cols = [c for c in required if c not in df.columns]
assert not missing_cols, f"Missing columns: {missing_cols}"
assert df[required].isnull().sum().sum() == 0, "Nulls found in required fields"

# --- 2) reconstruct task instances (robust pairing) ---
# Keep only task lifecycle rows (exclude gateways and START/COMPLETE for case)
task_rows = df[df["activity"].ne("START") & df["status"].isin(["queued","running"])].copy()

# Sort so cumcounts respect time
task_rows = task_rows.sort_values(["simulation_run","case_id","activity","timestamp"]).copy()

# Split and make occurrence indices *per status* (not mixed)
queued = task_rows[task_rows["status"]=="queued"].copy()
queued["occ_q"] = queued.groupby(["simulation_run","case_id","activity"]).cumcount()

running = task_rows[task_rows["status"]=="running"].copy()
running["occ_r"] = running.groupby(["simulation_run","case_id","activity"]).cumcount()

# Merge kth queued with kth running for each (simulation_run, case, activity)
inst = pd.merge(
    queued[["simulation_run","case_id","activity","occ_q","timestamp"]].rename(columns={"timestamp":"queued_ts"}),
    running[["simulation_run","case_id","activity","occ_r","timestamp","end_time","resource"]].rename(
        columns={"timestamp":"start_ts","end_time":"end_ts","resource":"resource_name"}),
    left_on=["simulation_run","case_id","activity","occ_q"],
    right_on=["simulation_run","case_id","activity","occ_r"],
    how="inner",
).drop(columns=["occ_q","occ_r"])

print("num task instances:", len(inst))
print("sample inst:\n", inst.head())

# Durations
inst["wait"] = inst["start_ts"] - inst["queued_ts"]
inst["service"] = inst["end_ts"] - inst["start_ts"]
inst["sojourn"] = inst["end_ts"] - inst["queued_ts"]

# Sanity: no negatives
neg = inst[(inst["wait"]<0) | (inst["service"]<0) | (inst["sojourn"]<0)]
assert neg.empty, f"Negative times found:\n{neg.head()}"

# --- 3) FIFO discipline per station (warning-free) ---
tmp = inst.sort_values(["simulation_run","activity","queued_ts","start_ts"]).copy()
eps = 1e-9
tmp["dec"] = tmp.groupby(["simulation_run","activity"])["start_ts"].diff() < -eps

fifo_check = (
    tmp.groupby(["simulation_run","activity"], as_index=False)
       .agg(num_instances=("start_ts","size"),
            fifo_violations=("dec","sum"))
)
fifo_check["fifo_violations"] = fifo_check["fifo_violations"].astype(int)

# --- 4) routing/QC probabilities (position-based, handles same-timestamp + END) ---

# Sort by simulation_run, case then time, and give each row an order within the case
df_sorted = df.sort_values(["simulation_run","case_id", "timestamp"]).copy()
df_sorted["row_in_case"] = df_sorted.groupby(["simulation_run","case_id"]).cumcount()

# Grab gateway rows (after they have row_in_case)
gates = df_sorted[df_sorted["status"] == "gateway"][["simulation_run","case_id","timestamp","activity","row_in_case"]].copy()
gates = gates.sort_values(["simulation_run","case_id","row_in_case"]).reset_index(drop=True)

def next_step_after_gateway(sim_run, case_id, row_idx_after):
    """
    Return the 'next activity' chosen after a gateway for a given case, by scanning
    the subsequent rows (in order) for that case:
      - first 'queued' row -> its activity is the chosen branch
      - if none and gateway was QC_AFTER_PACKAGING, treat next COMPLETE as END
    """
    sub = df_sorted[(df_sorted["simulation_run"] == sim_run) & 
                    (df_sorted["case_id"] == case_id) & 
                    (df_sorted["row_in_case"] > row_idx_after)]
    if sub.empty:
        return np.nan
    nxt_q = sub.loc[sub["status"] == "queued"]
    if not nxt_q.empty:
        return nxt_q.iloc[0]["activity"]
    # No queued afterwards; for packaging gateway the case may complete next
    nxt_c = sub.loc[sub["status"] == "COMPLETE"]
    if not nxt_c.empty:
        return "END"
    return np.nan

gates["next_activity"] = [
    next_step_after_gateway(sim_run, cid, r)
    for sim_run, cid, r in gates[["simulation_run","case_id","row_in_case"]].itertuples(index=False, name=None)
]

# Empirical routing per gateway label (aggregate across all simulation runs)
routing_emp = (
    gates.groupby("activity")["next_activity"]
         .value_counts(normalize=True, dropna=True)
         .rename("prob")
         .reset_index()
)

# Expected (from your config)
expected = {
    "QC_AFTER_MOULDING":  {"MOULDING": 0.05, "ASSEMBLY_1": 0.95},
    "QC_AFTER_ASSEMBLY1": {"ASSEMBLY_1": 0.05, "ASSEMBLY_2": 0.95},
    "QC_AFTER_ASSEMBLY2": {"ASSEMBLY_2": 0.05, "SORTING":    0.95},
    "QC_AFTER_SORTING":   {"SORTING":   0.05, "PACKAGING":   0.95},
    "QC_AFTER_PACKAGING": {"PACKAGING": 0.05, "END":         0.95},
}
routing_exp = pd.DataFrame(
    [{"activity": g, "next_activity": nxt, "expected": p}
     for g, dd in expected.items() for nxt, p in dd.items()]
)

routing = (routing_exp
           .merge(routing_emp, on=["activity","next_activity"], how="left")
           .fillna({"prob": 0.0}))
routing["diff"] = routing["prob"] - routing["expected"]

print("\nRouting empirical vs expected (position-based):")
print(routing.sort_values(["activity","next_activity"]))

# How many gateways had 0-wait to the next station?
merge_for_wait = gates.merge(
    inst[["simulation_run","case_id","activity","queued_ts","start_ts"]],
    left_on=["simulation_run","case_id","next_activity"], 
    right_on=["simulation_run","case_id","activity"],
    how="left", suffixes=("_gate","_next")
)
zero_wait_share = (merge_for_wait["start_ts"] - merge_for_wait["timestamp"] <= 1e-9).mean()
print("Share of gateways with immediate (zero-wait) handoff:", zero_wait_share)

# --- 5) service-time by resource ---
svc = inst.dropna(subset=["resource_name","service"]).copy()
svc_stats = svc.groupby("resource_name")["service"].agg(["count","mean","std","min","max"]).reset_index()

# Rough exponential check: for Exp, mean ≈ std; ratio near 1 is indicative (not proof)
svc_stats["std_over_mean"] = svc_stats["std"]/svc_stats["mean"]

# --- 6) utilization & throughput ---
# Calculate T per simulation run and average
T_per_run = df.groupby("simulation_run")["timestamp"].max()
T = T_per_run.mean()  # average simulation horizon

busy = (
    svc.groupby("resource_name", as_index=False)["service"]
       .sum()
       .rename(columns={"service":"busy_time"})
)
busy["utilization"] = busy["busy_time"] / (T * len(T_per_run))  # total busy time / (avg_T * num_runs)

# Station-level KPIs
station = inst.groupby("activity").agg(
    arrivals=("activity","size"),
    mean_wait=("wait","mean"),
    mean_service=("service","mean"),
    mean_sojourn=("sojourn","mean")
).reset_index()

# --- 7) Little's Law check: L ≈ λ W at station level ---
# λ = arrivals / T; W = mean_sojourn at station; L_hat = λ*W
station["lambda"] = station["arrivals"] / (T * len(T_per_run))
station["L_hat"] = station["lambda"] * station["mean_sojourn"]

# --- Summaries to print or save ---
print("\nFIFO violations per station per run (should be 0):")
print(fifo_check.groupby("activity")["fifo_violations"].sum().sort_values(ascending=False).head(10))

print("\nRouting empirical vs expected (diff near 0):")
print(routing.sort_values(["activity","next_activity"]))

print("\nService by resource (std/mean ~ 1 for Exp):")
print(svc_stats.sort_values("std_over_mean"))

print("\nResource utilization (sanity):")
print(busy.sort_values("utilization", ascending=False).head(10))

print("\nStation KPIs + Little's Law proxy:")
print(station.sort_values("L_hat", ascending=False).head(10))

# --- Stability audit: capacity vs offered load, with warm-up trim ---
warmup_fraction = 0.05

def analyze_stability_for_run(run_df, run_id):
    """Analyze stability for a single simulation run"""
    warmup = run_df["timestamp"].max() * warmup_fraction
    inst_run = inst[inst["simulation_run"] == run_id].copy()
    inst_ss = inst_run[inst_run["queued_ts"] >= warmup].copy()
    
    if inst_ss.empty:
        return None
        
    T_eff = run_df["timestamp"].max() - warmup
    
    # Per-station arrival rate λ_station (jobs/min)
    arrivals = inst_ss.groupby("activity")["activity"].size().rename("arrivals").reset_index()
    arrivals["lambda_station"] = arrivals["arrivals"] / T_eff
    
    # Mean service time per (activity, resource) then aggregate to station capacity
    svc_ss = inst_ss.dropna(subset=["resource_name","service"]).copy()
    if svc_ss.empty:
        return None
        
    rs = (svc_ss.groupby(["activity","resource_name"])["service"]
                .mean()
                .rename("mean_service")
                .reset_index())
    
    # Station capacity (jobs/min) = sum over resources of 1 / E[S_resource]
    cap = (rs.assign(mu=lambda d: 1.0 / d["mean_service"])
             .groupby("activity")["mu"].sum()
             .rename("capacity")
             .reset_index())
    
    # Merge and compute utilization proxy and safety margin
    stab = (arrivals.merge(cap, on="activity", how="left"))
    stab["rho_hat"] = stab["lambda_station"] / stab["capacity"]
    stab["headroom"] = 1.0 - stab["rho_hat"]
    stab["simulation_run"] = run_id
    
    return stab

# Analyze stability for each run and combine
stability_results = []
for run_id in df["simulation_run"].unique():
    run_df = df[df["simulation_run"] == run_id]
    stab = analyze_stability_for_run(run_df, run_id)
    if stab is not None:
        stability_results.append(stab)

if stability_results:
    combined_stability = pd.concat(stability_results, ignore_index=True)
    avg_stability = combined_stability.groupby("activity").agg({
        "rho_hat": "mean",
        "headroom": "mean",
        "lambda_station": "mean",
        "capacity": "mean"
    }).reset_index()
    
    print("\nAverage stability headroom by station across all runs:")
    print(avg_stability.sort_values("rho_hat", ascending=False))
else:
    print("\nNo stability data available")

# --- Queue trend analysis ---
def queue_trace(inst_df, act, run_id):
    run_data = inst_df[inst_df["simulation_run"] == run_id]
    g = run_data[run_data["activity"]==act][["queued_ts","start_ts"]].copy()
    if g.empty: 
        return None
    ups   = g.groupby("queued_ts").size().rename("delta")
    downs = (g.groupby("start_ts").size() * -1).rename("delta")
    deltas = pd.concat([ups, downs], axis=0).groupby(level=0).sum().sort_index()
    tr = deltas.cumsum().reset_index().rename(columns={"index":"time","delta":"qlen"})
    tr["qlen"] = tr["qlen"].clip(lower=0)
    return tr

def end_trend_slope(trace, tail_frac=0.2):
    if trace is None or len(trace) < 5: 
        return np.nan
    n0 = int(len(trace)*(1-tail_frac))
    tail = trace.iloc[max(n0,0):].copy()
    if len(tail) < 3: 
        return np.nan
    # simple least-squares slope of qlen on time
    x = tail["time"].values
    y = tail["qlen"].values
    A = np.vstack([x, np.ones_like(x)]).T
    slope, _ = np.linalg.lstsq(A, y, rcond=None)[0]
    return slope  # jobs per minute near the end

# Analyze queue trends per run and activity
trend_results = []
for run_id in df["simulation_run"].unique():
    run_inst = inst[inst["simulation_run"] == run_id]
    warmup = run_inst["queued_ts"].max() * warmup_fraction
    inst_ss_run = run_inst[run_inst["queued_ts"] >= warmup]
    
    acts = sorted(inst_ss_run["activity"].unique())
    for act in acts:
        tr = queue_trace(inst_ss_run, act, run_id)
        slope = end_trend_slope(tr, tail_frac=0.3)
        trend_results.append({
            "simulation_run": run_id, 
            "activity": act, 
            "end_slope_q_per_min": slope
        })

trend_df = pd.DataFrame(trend_results)
avg_trend = trend_df.groupby("activity")["end_slope_q_per_min"].mean().reset_index()

print("\nAverage queue end-trend slope across all runs (jobs/min; ≈0 is good):")
print(avg_trend.sort_values("end_slope_q_per_min", ascending=False))

num task instances: 72378
sample inst:
    simulation_run  case_id    activity  queued_ts   start_ts     end_ts  \
0               0        0  ASSEMBLY_1  20.393651  20.393651  24.866156   
1               0        0  ASSEMBLY_2  24.866156  24.866156  26.676780   
2               0        0    MOULDING  18.537263  18.537263  20.393651   
3               0        0   PACKAGING  29.063644  29.063644  36.031979   
4               0        0     SORTING  26.676780  26.676780  29.063644   

        resource_name  
0    LINE_1_ASSEMBLY1  
1    ASSEMBLY2_LINE_1  
2  MOULDING_MACHINE_1  
3    PACKAGING_LINE_1  
4       SORTING_ROBOT  

Routing empirical vs expected (position-based):
             activity next_activity  expected      prob      diff
2  QC_AFTER_ASSEMBLY1    ASSEMBLY_1      0.05  0.050476  0.000476
3  QC_AFTER_ASSEMBLY1    ASSEMBLY_2      0.95  0.949524 -0.000476
4  QC_AFTER_ASSEMBLY2    ASSEMBLY_2      0.05  0.049473 -0.000527
5  QC_AFTER_ASSEMBLY2       SORTING      0.95  0.950

In [ ]:
# =========================
# Inductive Miner + DFG (PM4Py)
# End-to-end discovery from simulator CSVs
# =========================

# --- installs (safe to rerun) ---
import sys, subprocess, pkgutil
def _pip(pkg):
    if pkg not in {m.name for m in pkgutil.iter_modules()}:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["pm4py", "graphviz"]:
    try:
        _pip(p)
    except Exception as e:
        print(f"[WARN] Could not auto-install {p}: {e}")

# --- imports ---
import os, glob
import pandas as pd
import numpy as np

from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.visualization.petri_net import visualizer as pn_vis

from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_vis

# =========================
# CONFIG
# =========================
LOGS_GLOB        = "results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv"   # <-- adjust if your files live elsewhere
SCENARIO_FILTER  = None              # e.g., "actuator_manufacturing_with_rework" or None
USE_ALL_RUNS     = True              # use all runs for discovery except the held-out
HOLDOUT_IS_LAST  = True              # treat the last run id as held-out (ignored if not splitting)
HOLDOUT_RUN      = None              # e.g., 6  (used if HOLDOUT_IS_LAST == False)
WARMUP_FRAC      = 0.10              # whole-case trim: drop cases starting before 10% of run span
COOLDOWN_FRAC    = 0.02              # drop cases ending after last 2% of span
EPS              = 1e-9              # tiny epsilon for time comparisons

OUT_DIR          = "results"
os.makedirs(OUT_DIR, exist_ok=True)

# =========================
# LOAD CSVs
# =========================
files = sorted(glob.glob(LOGS_GLOB))
if not files:
    raise FileNotFoundError(f"No CSVs matched {LOGS_GLOB}")

dfs = []
for f in files:
    d = pd.read_csv(f)
    # required minimal columns
    required = {"timestamp","status","case_id","activity"}
    missing = required - set(d.columns)
    if missing:
        raise ValueError(f"{f} is missing required columns: {missing}")
    if SCENARIO_FILTER and "scenario" in d.columns:
        d = d[d["scenario"] == SCENARIO_FILTER]
        if d.empty:
            continue
    dfs.append(d)

if not dfs:
    raise RuntimeError("After filtering, no logs remain.")

df = pd.concat(dfs, ignore_index=True)

# parse numeric time columns
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
if "end_time" in df.columns:
    df["end_time"] = pd.to_numeric(df["end_time"], errors="coerce")

# ensure simulation_run exists
if "simulation_run" not in df.columns:
    df["simulation_run"] = 0

all_runs = sorted(df["simulation_run"].dropna().unique().tolist())
if not all_runs:
    all_runs = [0]

if HOLDOUT_IS_LAST:
    HOLDOUT_RUN = all_runs[-1]
elif HOLDOUT_RUN is None:
    HOLDOUT_RUN = all_runs[-1]

disc_runs = [r for r in all_runs if r != HOLDOUT_RUN] if USE_ALL_RUNS else [all_runs[0]]
held_runs = [HOLDOUT_RUN]

print(f"[Info] runs found: {all_runs}")
print(f"[Info] discovery runs: {disc_runs} | held-out run: {held_runs}")

# keep only columns we need downstream
keep_cols = [c for c in ["simulation_run","timestamp","status","case_id","activity","resource","end_time"] if c in df.columns]
df = df[keep_cols].copy()

# =========================
# WHOLE-CASE TRIM (per run)
# =========================
def trim_cases_whole(df_run, warmup_frac=WARMUP_FRAC, cooldown_frac=COOLDOWN_FRAC):
    t_max = df_run["timestamp"].max()
    t0 = t_max * warmup_frac
    t1 = t_max * (1.0 - cooldown_frac)

    starts = (df_run[df_run["status"]=="START"]
              .groupby("case_id")["timestamp"].min().rename("case_start"))
    last   = df_run.groupby("case_id")["timestamp"].max().rename("case_last")
    ends   = (df_run[df_run["status"]=="COMPLETE"]
              .groupby("case_id")["timestamp"].max().rename("case_end"))
    meta = pd.concat([starts, ends, last], axis=1).reset_index()
    meta["case_end_fallback"] = meta["case_end"].fillna(meta["case_last"])

    keep_ids = meta[(meta["case_start"] >= t0) & (meta["case_end_fallback"] <= t1)]["case_id"]
    return df_run[df_run["case_id"].isin(keep_ids)].copy()

def trim_all_runs(df_any):
    pieces = []
    for r in sorted(df_any["simulation_run"].unique()):
        dr = df_any[df_any["simulation_run"]==r].copy()
        pieces.append(trim_cases_whole(dr))
    return pd.concat(pieces, ignore_index=True)

df_disc_raw = df[df["simulation_run"].isin(disc_runs)].copy()
df_hold_raw = df[df["simulation_run"].isin(held_runs)].copy()
df_disc = trim_all_runs(df_disc_raw)
df_hold = trim_all_runs(df_hold_raw)

print(f"[Info] cases after trim — discovery: {df_disc['case_id'].nunique()}, held-out: {df_hold['case_id'].nunique()}")

# =========================
# RECONSTRUCT TASK INSTANCES (queued → running)
# =========================
def pair_instances(df_run):
    # keep only task lifecycle rows (exclude gateway + START/COMPLETE)
    mask = (df_run["activity"].ne("START")) & (df_run["status"].isin(["queued","running"]))
    task_rows = df_run[mask].copy()
    task_rows = task_rows.sort_values(["simulation_run","case_id","activity","timestamp"])

    queued  = task_rows[task_rows["status"]=="queued"].copy()
    queued["occ_q"] = queued.groupby(["simulation_run","case_id","activity"]).cumcount()

    running = task_rows[task_rows["status"]=="running"].copy()
    running["occ_r"] = running.groupby(["simulation_run","case_id","activity"]).cumcount()

    inst = pd.merge(
        queued[["simulation_run","case_id","activity","occ_q","timestamp"]],
        running[["simulation_run","case_id","activity","occ_r","timestamp","end_time","resource"]],
        left_on=["simulation_run","case_id","activity","occ_q"],
        right_on=["simulation_run","case_id","activity","occ_r"],
        how="inner",
        suffixes=("_queued","_start")
    ).drop(columns=["occ_q","occ_r"])

    inst = inst.rename(columns={
        "timestamp_queued":"queued_ts",
        "timestamp_start":"start_ts",
        "end_time":"end_ts",
        "resource":"org:resource"
    })
    # durations (sanity)
    inst["wait"]    = inst["start_ts"] - inst["queued_ts"]
    inst["service"] = inst["end_ts"]   - inst["start_ts"]
    inst["sojourn"] = inst["end_ts"]   - inst["queued_ts"]
    inst = inst[(inst["wait"]>=-EPS) & (inst["service"]>=-EPS) & (inst["sojourn"]>=-EPS)].copy()
    return inst

inst_disc = pair_instances(df_disc)
inst_hold = pair_instances(df_hold)

print(f"[Info] instances — discovery: {len(inst_disc)} | held-out: {len(inst_hold)}")

# =========================
# BUILD PM4Py EVENT LOG with lifecycle
# =========================
def build_event_df(inst, df_raw):
    """Build event log including START/END boundaries"""
    
    # 1) Task lifecycle events (start and complete)
    start_events = inst.rename(columns={
        "start_ts":"time:timestamp",
        "activity":"concept:name"
    })[["simulation_run","case_id","concept:name","time:timestamp","org:resource"]].copy()
    start_events["lifecycle:transition"] = "start"

    complete_events = inst.rename(columns={
        "end_ts":"time:timestamp",
        "activity":"concept:name"
    })[["simulation_run","case_id","concept:name","time:timestamp","org:resource"]].copy()
    complete_events["lifecycle:transition"] = "complete"

    # 2) Add START events (case arrival) - these mark the beginning of each case
    start_rows = df_raw[df_raw["status"] == "START"].copy()
    start_rows = start_rows.rename(columns={
        "timestamp": "time:timestamp",
        "activity": "concept:name"
    })
    start_rows["lifecycle:transition"] = "complete"  # START is instantaneous
    start_rows["org:resource"] = None
    start_rows = start_rows[["simulation_run","case_id","concept:name","time:timestamp","org:resource","lifecycle:transition"]]

    # 3) Add END events (case completion) - these mark the end of each case
    end_rows = df_raw[df_raw["status"] == "COMPLETE"].copy()
    end_rows = end_rows.rename(columns={
        "timestamp": "time:timestamp",
        "activity": "concept:name"  
    })
    end_rows["lifecycle:transition"] = "complete"
    end_rows["org:resource"] = None
    end_rows = end_rows[["simulation_run","case_id","concept:name","time:timestamp","org:resource","lifecycle:transition"]]

    # 4) Combine all events
    ev = pd.concat([start_rows, start_events, complete_events, end_rows], ignore_index=True)
    
    # PM4Py keys
    ev = ev.rename(columns={"case_id":"case:concept:name"})
    ev["trace_id"] = ev["simulation_run"].astype(str) + "-" + ev["case:concept:name"].astype(str)
    
    # Sort by case and timestamp to ensure proper ordering
    ev = ev.sort_values(["trace_id", "time:timestamp"]).reset_index(drop=True)
    
    # Convert timestamps
    ev["time:timestamp"] = pd.to_datetime(ev["time:timestamp"], unit="m", origin="unix", errors="coerce")
    
    return ev

ev_disc = build_event_df(inst_disc, df_disc)
ev_hold = build_event_df(inst_hold, df_hold)

# Save CSV event logs (optional, useful to inspect)
ev_disc.to_csv(os.path.join(OUT_DIR, "discovery_eventlog.csv"), index=False)
ev_hold.to_csv(os.path.join(OUT_DIR, "heldout_eventlog.csv"), index=False)
print("[Saved] discovery_eventlog.csv, heldout_eventlog.csv")

# PM4Py event log objects
params = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: "trace_id"}
log_disc = log_converter.apply(ev_disc, variant=log_converter.Variants.TO_EVENT_LOG, parameters=params)
log_hold = log_converter.apply(ev_hold, variant=log_converter.Variants.TO_EVENT_LOG, parameters=params)

# =========================
# INDUCTIVE MINER (IMf) -> convert ProcessTree to Petri net
# =========================
print("[Discovery] Inductive Miner (IMf)…")
from pm4py.objects.conversion.process_tree import converter as pt_converter

# 1) Discover process tree
ptree = inductive_miner.apply(log_disc, variant=inductive_miner.Variants.IMf)
print("[Info] Inductive Miner returned:", type(ptree).__name__)  # should be 'ProcessTree'

# 2) Convert tree -> Petri net
net_i, im_i, fm_i = pt_converter.apply(ptree, variant=pt_converter.Variants.TO_PETRI_NET)

# 3) Save PNML and PNG
pnml_ind = os.path.join(OUT_DIR, "model_inductive.pnml")
pnml_exporter.apply(net_i, im_i, pnml_ind, final_marking=fm_i)
print(f"[Saved] {pnml_ind}")

try:
    gviz_i = pn_vis.apply(net_i, im_i, fm_i)
    pn_vis.save(gviz_i, os.path.join(OUT_DIR, "model_inductive.png"))
    print("[Saved] model_inductive.png")
except Exception as e:
    print(f"[WARN] Could not render Petri net PNG (Graphviz missing/not on PATH): {e}")


# =========================
# DFG (frequency + performance)
# =========================
print("[Discovery] DFG (frequency + performance)…")

# frequency DFG
dfg_freq = dfg_discovery.apply(log_disc, variant=dfg_discovery.Variants.FREQUENCY)
# performance DFG (edge performance as mean time between activities)
dfg_perf = dfg_discovery.apply(log_disc, variant=dfg_discovery.Variants.PERFORMANCE)

# visualize frequency DFG
try:
    gviz_dfg = dfg_vis.apply(dfg_freq, log_disc, variant=dfg_vis.Variants.FREQUENCY)
    dfg_vis.save(gviz_dfg, os.path.join(OUT_DIR, "dfg_frequency.png"))
    print("[Saved] dfg_frequency.png")
except Exception as e:
    print(f"[WARN] Could not render DFG PNG (Graphviz missing/not on PATH): {e}")

# export DFG edges to CSV (frequency + performance)
def dfg_to_frame(dfg_dict, col_name):
    rows = []
    for (a,b), val in dfg_dict.items():
        rows.append({"from": a, "to": b, col_name: val})
    return pd.DataFrame(rows)

dfg_freq_df = dfg_to_frame(dfg_freq, "frequency")
dfg_perf_df = dfg_to_frame(dfg_perf, "mean_time")  # PM4Py stores average durations here

dfg_edges = pd.merge(dfg_freq_df, dfg_perf_df, on=["from","to"], how="outer")
dfg_edges = dfg_edges.sort_values(["from","to"]).reset_index(drop=True)
dfg_edges.to_csv(os.path.join(OUT_DIR, "dfg_edges.csv"), index=False)
print("[Saved] dfg_edges.csv  (freq + mean_time)")

# quick summary
print("\n=== Summary ===")
print(f"Traces (discovery): {len(log_disc)} | Traces (held-out): {len(log_hold)}")
print(f"Activities (discovery): {ev_disc['concept:name'].nunique()}")
print(f"DFG edges: {len(dfg_edges)}")
print("Files saved in:", os.path.abspath(OUT_DIR))


[Info] runs found: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[Info] discovery runs: [0, 1, 2, 3, 4, 5, 6, 7, 8] | held-out run: [9]
[Info] cases after trim — discovery: 1292, held-out: 1225
[Info] instances — discovery: 56749 | held-out: 6450
[Saved] discovery_eventlog.csv, heldout_eventlog.csv
[Discovery] Inductive Miner (IMf)…
[Info] Inductive Miner returned: ProcessTree
[Saved] results\model_inductive.pnml
[Saved] model_inductive.png
[Discovery] DFG (frequency + performance)…
[Saved] dfg_frequency.png
[Saved] dfg_edges.csv  (freq + mean_time)

=== Summary ===
Traces (discovery): 10766 | Traces (held-out): 1225
Activities (discovery): 5
DFG edges: 10
Files saved in: c:\Users\990215322\Desktop\MuProMAC\results


In [ ]:
import os
os.environ["PM4PY_DISABLE_CVXOPT"] = "1"
os.environ["PM4PY_LP_SOLVER"] = "scipy"

import pm4py
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from pm4py.objects.conversion.log import converter as xes_converter

PORTABLE = r"C:\Users\990215322\Downloads\windows_10_cmake_Release_Graphviz-14.0.2-win64 (1)\Graphviz-14.0.2-win64"
os.environ["PATH"] = PORTABLE + r"\bin;" + os.environ.get("PATH", "")
os.environ["GV_PLUGIN_PATH"] = PORTABLE + r"\lib\graphviz"
os.environ["GRAPHVIZ_DOT"] = PORTABLE + r"\bin\dot.exe"

os.makedirs("process_visualizations", exist_ok=True)

# --- quick Graphviz smoke test ---
try:
    import graphviz as _gv
    _gv.Source("digraph{A->B}").render("process_visualizations/_graphviz_ok", format="png", cleanup=True)
    print("[OK] Graphviz reachable (created process_visualizations/_graphviz_ok.png)")
except Exception as e:
    print("[WARN] Graphviz smoke test failed:", e)

class ProcessMiner:
    def __init__(self, df):
        self.df = df
        self.event_log = None
        self.models = {}
        
    def prepare_event_log(self, run_id=None):
        """
        Build one event per activity instance (complete at end_ts).
        This avoids the raw status confusion (queued/running/COMPLETE).
        """
        df = self.df.copy()
        if run_id is not None:
            df = df[df['simulation_run'] == run_id].copy()

        # --------- 1) robust instance reconstruction ----------
        case_col_src = 'case_id'
        task_rows = df[
            (df['status'].isin(['queued', 'running'])) &
            (df['activity'].ne('START')) &
            (df['activity'].ne('END'))
        ].copy()

        task_rows = task_rows.sort_values([case_col_src, 'activity', 'timestamp'])

        queued = (task_rows[task_rows['status'] == 'queued']
                .assign(idx=lambda d: d.groupby([case_col_src, 'activity']).cumcount())
                [[case_col_src, 'activity', 'timestamp', 'idx']]
                .rename(columns={'timestamp': 'queued_ts'}))

        running = (task_rows[task_rows['status'] == 'running']
                .assign(idx=lambda d: d.groupby([case_col_src, 'activity']).cumcount())
                [[case_col_src, 'activity', 'timestamp', 'end_time', 'resource', 'idx']]
                .rename(columns={'timestamp': 'start_ts',
                                    'end_time': 'end_ts',
                                    'resource': 'org:resource'}))

        # Keep all running (completed services); fill missing queued as zero-wait
        inst = pd.merge(running, queued, on=[case_col_src, 'activity', 'idx'], how='left')
        inst['queued_ts'] = inst['queued_ts'].fillna(inst['start_ts'])

        # sanity: drop negatives
        inst = inst[inst['end_ts'] >= inst['start_ts']].copy()

        # If we are using multiple runs, make cross-run-unique case id
        if run_id is None:
            inst['case:concept:name'] = (
                df['simulation_run'].astype(str) + '_' + inst[case_col_src].astype(str)
            )
        else:
            inst['case:concept:name'] = inst[case_col_src].astype(str)

        # --------- 2) build an event log of COMPLETED activities ----------
        event_log_df = inst.rename(columns={
            'activity': 'concept:name',
            'end_ts': 'time:timestamp'
        })[['case:concept:name', 'concept:name', 'time:timestamp', 'org:resource']].copy()

        # timestamps are minutes-from-zero; map to datetime
        event_log_df['time:timestamp'] = pd.to_datetime(
            event_log_df['time:timestamp'], unit='m', origin='2024-01-01'
        )
        event_log_df['case:concept:name'] = event_log_df['case:concept:name'].astype(str)

        # --------- 3) convert to PM4Py EventLog (fallback to DF) ----------
        try:
            from pm4py.objects.conversion.log import converter as xes_converter
            self.event_log = xes_converter.apply(
                event_log_df, variant=xes_converter.Variants.TO_EVENT_LOG
            )
            n_cases = len({ev['case:concept:name'] for ev in self.event_log})
            print(f"📊 Prepared event log: {len(self.event_log)} events, {n_cases} cases")
        except Exception as e:
            print(f"⚠️ Event log conversion failed: {e}")
            self.event_log = event_log_df
            print(f"📊 Using DataFrame directly: {len(self.event_log)} events, {event_log_df['case:concept:name'].nunique()} cases")

        return self.event_log
    def discover_dfg(self):
        """Discover Directly-Follows Graph (frequency-based) with starts/ends."""
        print("🔄 Discovering Directly-Follows Graph...")
        try:
            # PM4Py DFG discovery (works for EventLog or pandas DF with pm4py column names)
            from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
            from pm4py.algo.discovery.dfg.adapters.pandas import df_statistics as dfg_stats_df
            from pm4py.statistics.start_activities.pandas import get as start_acts_df
            from pm4py.statistics.end_activities.pandas import get as end_acts_df

            if isinstance(self.event_log, pd.DataFrame):
                # Pandas path
                dfg = dfg_stats_df.get_dfg_graph(self.event_log, case_id_glue="case:concept:name",
                                                 activity_key="concept:name", timestamp_key="time:timestamp")
                start_activities = start_acts_df.get_start_activities(self.event_log,
                                                                      case_id_glue="case:concept:name",
                                                                      activity_key="concept:name",
                                                                      timestamp_key="time:timestamp")
                end_activities = end_acts_df.get_end_activities(self.event_log,
                                                                case_id_glue="case:concept:name",
                                                                activity_key="concept:name",
                                                                timestamp_key="time:timestamp")
            else:
                # EventLog path
                dfg = dfg_discovery.apply(self.event_log)
                from pm4py.statistics.start_activities.log import get as start_acts_log
                from pm4py.statistics.end_activities.log import get as end_acts_log
                start_activities = start_acts_log.get_start_activities(self.event_log)
                end_activities = end_acts_log.get_end_activities(self.event_log)

        except Exception as e:
            print(f"⚠️ DFG discovery via PM4Py failed: {e}. Falling back to manual.")
            dfg, start_activities, end_activities = self._calculate_dfg_manually()

        # Probabilities
        total = sum(dfg.values()) if isinstance(dfg, dict) else 0
        dfg_probabilities = {k: v / total for k, v in dfg.items()} if total > 0 else {}

        activities = list(set(a for edge in dfg.keys() for a in edge)) if dfg else list(
            set(list(start_activities.keys()) + list(end_activities.keys()))
        )

        model = {
            "type": "dfg",
            "dfg": dfg,
            "dfg_probabilities": dfg_probabilities,
            "start_activities": start_activities,
            "end_activities": end_activities,
            "activities": activities,
        }
        self.models["dfg"] = model
        print(f"✅ DFG discovered: {len(dfg)} relations | "
              f"starts: {len(start_activities)} | ends: {len(end_activities)}")
        return model


    def _calculate_dfg_manually(self):
        """Manual fallback for DFG calculation"""
        dfg = {}
        start_activities = {}
        end_activities = {}
        
        # Handle both DataFrame and EventLog
        if isinstance(self.event_log, pd.DataFrame):
            # DataFrame approach
            cases = self.event_log.groupby('case:concept:name')
            for case_id, case_events in cases:
                case_events_sorted = case_events.sort_values('time:timestamp')
                
                # Record start activity
                if len(case_events_sorted) > 0:
                    start_act = case_events_sorted.iloc[0]['concept:name']
                    start_activities[start_act] = start_activities.get(start_act, 0) + 1
                
                # Record DFG relations
                for i in range(len(case_events_sorted) - 1):
                    from_act = case_events_sorted.iloc[i]['concept:name']
                    to_act = case_events_sorted.iloc[i + 1]['concept:name']
                    dfg[(from_act, to_act)] = dfg.get((from_act, to_act), 0) + 1
                
                # Record end activity
                if len(case_events_sorted) > 0:
                    end_act = case_events_sorted.iloc[-1]['concept:name']
                    end_activities[end_act] = end_activities.get(end_act, 0) + 1
        else:
            # EventLog approach
            cases = {}
            for event in self.event_log:
                case_id = event['case:concept:name']
                if case_id not in cases:
                    cases[case_id] = []
                cases[case_id].append(event)
            
            # Sort events by timestamp within each case
            for case_id, events in cases.items():
                events_sorted = sorted(events, key=lambda x: x['time:timestamp'])
                
                # Record start activity
                if events_sorted:
                    start_act = events_sorted[0]['concept:name']
                    start_activities[start_act] = start_activities.get(start_act, 0) + 1
                
                # Record DFG relations
                for i in range(len(events_sorted) - 1):
                    from_act = events_sorted[i]['concept:name']
                    to_act = events_sorted[i + 1]['concept:name']
                    dfg[(from_act, to_act)] = dfg.get((from_act, to_act), 0) + 1
                
                # Record end activity
                if events_sorted:
                    end_act = events_sorted[-1]['concept:name']
                    end_activities[end_act] = end_activities.get(end_act, 0) + 1
        
        return dfg, start_activities, end_activities

    def discover_inductive_miner(self, noise_threshold=0.2):
        """Discover process model using Inductive Miner"""
        print("🔄 Discovering process with Inductive Miner...")
        
        from pm4py.algo.discovery.inductive import algorithm as inductive_miner
        
        try:
            # Use IMf variant which handles noise better
            tree = inductive_miner.apply(self.event_log, variant=inductive_miner.Variants.IMf, 
                                       parameters={'noise_threshold': noise_threshold})
            net, initial_marking, final_marking = pm4py.convert_to_petri_net(tree)
            
            model = {
                'type': 'inductive',
                'tree': tree,
                'petri_net': net,
                'initial_marking': initial_marking,
                'final_marking': final_marking
            }
            
            self.models['inductive'] = model
            print("✅ Inductive Miner model discovered")
            return model
        except Exception as e:
            print(f"⚠️ Inductive Miner failed: {e}")
            return None

    def discover_heuristics_miner(self, dependency_threshold=0.5):
        """Discover process model using Heuristics Miner"""
        print("🔄 Discovering process with Heuristics Miner...")
        
        from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
        
        try:
            parameters = {
                heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: dependency_threshold,
                heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_ACT_COUNT: 1
            }
            
            heu_net = heuristics_miner.apply(self.event_log, parameters=parameters)
            
            model = {
                'type': 'heuristics',
                'heu_net': heu_net
            }
            
            self.models['heuristics'] = model
            print("✅ Heuristics Miner model discovered")
            return model
        except Exception as e:
            print(f"⚠️ Heuristics Miner failed: {e}")
            return None

    def analyze_process_variants(self):
        """Analyze process variants in the event log"""
        print("🔍 Analyzing process variants...")
        
        try:
            variants = pm4py.get_variants_as_tuples(self.event_log)
            
            print(f"📋 Found {len(variants)} unique process variants")
            print("\nTop 10 most frequent variants:")
            
            for i, (variant, count) in enumerate(sorted(variants.items(), key=lambda x: x[1], reverse=True)[:10]):
                print(f"{i+1:2d}. Count: {count:4d} - {' → '.join(variant)}")
            
            return variants
        except Exception as e:
            print(f"⚠️ Variant analysis failed: {e}")
            return {}

    def analyze_process_metrics(self):
        """Calculate key process metrics"""
        print("📊 Calculating process metrics...")
        
        metrics = {}
        
        try:
            # Case statistics
            case_durations = pm4py.get_all_case_durations(self.event_log)
            metrics['case_durations'] = {
                'mean': np.mean(case_durations),
                'median': np.median(case_durations),
                'min': np.min(case_durations),
                'max': np.max(case_durations),
                'std': np.std(case_durations)
            }
            
            # Activity frequency - handle both DataFrame and EventLog
            if isinstance(self.event_log, pd.DataFrame):
                activity_counts = self.event_log['concept:name'].value_counts().to_dict()
            else:
                activity_counts = {}
                for event in self.event_log:
                    activity = event['concept:name']
                    activity_counts[activity] = activity_counts.get(activity, 0) + 1
            metrics['activity_frequency'] = activity_counts
            
            # Throughput calculation
            if isinstance(self.event_log, pd.DataFrame):
                start_times = self.event_log.groupby('case:concept:name')['time:timestamp'].min()
                end_times = self.event_log.groupby('case:concept:name')['time:timestamp'].max()
                total_time = (end_times.max() - start_times.min()).total_seconds() / 3600
                case_count = self.event_log['case:concept:name'].nunique()
            else:
                timestamps = [event['time:timestamp'] for event in self.event_log]
                if timestamps:
                    total_time = (max(timestamps) - min(timestamps)).total_seconds() / 3600
                    case_count = len(set(event['case:concept:name'] for event in self.event_log))
                else:
                    total_time = 0
                    case_count = 0
            
            metrics['throughput'] = case_count / total_time if total_time > 0 else 0
            
            print("\n📈 Process Metrics:")
            print(f"   Cases: {len(case_durations)}")
            print(f"   Average case duration: {metrics['case_durations']['mean']:.2f} minutes")
            print(f"   Throughput: {metrics['throughput']:.2f} cases/hour")
            print(f"   Unique activities: {len(activity_counts)}")
            
            print("\n🏭 Most frequent activities:")
            for activity, count in sorted(activity_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"   {activity}: {count} occurrences")
            
        except Exception as e:
            print(f"⚠️ Metrics calculation failed: {e}")
            metrics = {'error': str(e)}
        
        return metrics

    def compare_discovery_methods(self):
        """Compare characteristics of different discovery methods"""
        print("⚖️ Comparing discovery methods...")
        
        comparison = {}
        
        # DFG characteristics
        if 'dfg' in self.models:
            dfg_model = self.models['dfg']
            comparison['dfg'] = {
                'num_activities': len(dfg_model['activities']),
                'num_relations': len(dfg_model['dfg']),
                'start_activities': len(dfg_model['start_activities']),
                'end_activities': len(dfg_model['end_activities']),
                'description': 'Directly-Follows Graph - frequency-based relations'
            }
        
        # Inductive Miner characteristics
        if 'inductive' in self.models and self.models['inductive'] is not None:
            inductive_model = self.models['inductive']
            net = inductive_model['petri_net']
            comparison['inductive'] = {
                'num_places': len(net.places),
                'num_transitions': len(net.transitions),
                'num_arcs': len(net.arcs),
                'model_complexity': len(net.places) + len(net.transitions),
                'description': 'Inductive Miner - sound process tree converted to Petri net'
            }
        
        # Heuristics Miner characteristics
        if 'heuristics' in self.models and self.models['heuristics'] is not None:
            comparison['heuristics'] = {
                'description': 'Heuristics Net - dependency-based with frequency thresholds'
            }
        
        print("\n🔬 Discovery Method Comparison:")
        for method, chars in comparison.items():
            print(f"\n{method.upper()}:")
            for key, value in chars.items():
                print(f"  {key}: {value}")
        
        return comparison

    def save_discovered_models(self, output_dir="discovered_models"):
        """Save discovered models to files"""
        import os
        os.makedirs(output_dir, exist_ok=True)
        
        print(f"💾 Saving models to {output_dir}/...")
        
        # Save DFG as CSV
        if 'dfg' in self.models:
            dfg_data = []
            for (from_act, to_act), count in self.models['dfg']['dfg'].items():
                dfg_data.append({
                    'from_activity': from_act,
                    'to_activity': to_act,
                    'frequency': count,
                    'probability': self.models['dfg']['dfg_probabilities'].get((from_act, to_act), 0)
                })
            dfg_df = pd.DataFrame(dfg_data)
            dfg_df.to_csv(f"{output_dir}/dfg_relations.csv", index=False)
            print("✅ Saved DFG relations to dfg_relations.csv")
        
        # Save Petri net
        if 'inductive' in self.models and self.models['inductive'] is not None:
            try:
                pm4py.write_pnml(self.models['inductive']['petri_net'],
                               self.models['inductive']['initial_marking'],
                               self.models['inductive']['final_marking'],
                               f"{output_dir}/inductive_model.pnml")
                print("✅ Saved Inductive Miner model to inductive_model.pnml")
            except Exception as e:
                print(f"⚠️ Failed to save Inductive Miner model: {e}")
        
        # Save event log in XES format
        try:
            pm4py.write_xes(self.event_log, f"{output_dir}/event_log.xes")
            print("✅ Saved event log to event_log.xes")
        except Exception as e:
            print(f"⚠️ Failed to save event log: {e}")

    # ⭐⭐⭐ MAKE SURE THESE ARE PROPERLY INDENTED INSIDE THE CLASS ⭐⭐⭐
    def save_all_visualizations(self, output_dir="process_visualizations"):
        """Save all visualizations as image files"""
        os.makedirs(output_dir, exist_ok=True)
        
        print("💾 Saving visualizations as images...")
        
        try:
            # Save DFG
            if 'dfg' in self.models:
                from pm4py.visualization.dfg import visualizer as dfg_visualizer
                gviz = dfg_visualizer.apply(self.models['dfg']['dfg'], 
                                          self.models['dfg']['start_activities'],
                                          self.models['dfg']['end_activities'])
                dfg_visualizer.save(gviz, f"{output_dir}/dfg.png")
                print("✅ Saved DFG as dfg.png")
            
            # Save Process Tree
            if 'inductive' in self.models and self.models['inductive'] is not None:
                from pm4py.visualization.process_tree import visualizer as pt_visualizer
                gviz = pt_visualizer.apply(self.models['inductive']['tree'])
                pt_visualizer.save(gviz, f"{output_dir}/process_tree.png")
                print("✅ Saved Process Tree as process_tree.png")
                
                # Save Petri Net
                from pm4py.visualization.petri_net import visualizer as pn_visualizer
                gviz = pn_visualizer.apply(self.models['inductive']['petri_net'],
                                         self.models['inductive']['initial_marking'],
                                         self.models['inductive']['final_marking'])
                pn_visualizer.save(gviz, f"{output_dir}/petri_net.png")
                print("✅ Saved Petri Net as petri_net.png")
            
            # Save Heuristics Net
            if 'heuristics' in self.models and self.models['heuristics'] is not None:
                from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
                gviz = hn_visualizer.apply(self.models['heuristics']['heu_net'])
                hn_visualizer.save(gviz, f"{output_dir}/heuristics_net.png")
                print("✅ Saved Heuristics Net as heuristics_net.png")
                
        except Exception as e:
            print(f"⚠️ PM4Py visualization failed: {e}")
            print("🔄 Trying alternative approach...")
            self._create_visualizations_directly(output_dir)

    def _create_visualizations_directly(self, output_dir):
        """Create visualizations without relying on PM4Py's EventLog"""
        print("🎨 Creating visualizations using graphviz directly...")
        
        try:
            # 1. Create DFG visualization using graphviz directly
            if 'dfg' in self.models:
                import graphviz
                
                dot = graphviz.Digraph(comment='Manufacturing Process DFG')
                dot.attr(rankdir='LR', size='8,5')
                
                # Add nodes (activities)
                for activity in self.models['dfg']['activities']:
                    dot.node(activity, activity)
                
                # Add edges (DFG relations) with weights
                for (from_act, to_act), count in self.models['dfg']['dfg'].items():
                    # Calculate line thickness based on frequency
                    thickness = max(1, min(5, count / 100))
                    dot.edge(from_act, to_act, label=str(count), penwidth=str(thickness))
                
                dot.render(f'{output_dir}/dfg', format='png', cleanup=True)
                print("✅ Saved DFG as dfg.png")
            
            # 2. Create a simple process flow diagram
            if 'dfg' in self.models:
                import graphviz
                
                # Create a cleaner version showing main flow
                dot = graphviz.Digraph(comment='Manufacturing Main Flow')
                dot.attr(rankdir='LR', size='8,5')
                
                # Define the expected main flow
                main_flow = ['MOULDING', 'ASSEMBLY_1', 'ASSEMBLY_2', 'SORTING', 'PACKAGING', 'END']
                
                # Add nodes in order
                for activity in main_flow:
                    if activity in self.models['dfg']['activities']:
                        dot.node(activity, activity)
                
                # Connect main flow
                for i in range(len(main_flow) - 1):
                    if main_flow[i] in self.models['dfg']['activities'] and main_flow[i+1] in self.models['dfg']['activities']:
                        count = self.models['dfg']['dfg'].get((main_flow[i], main_flow[i+1]), 0)
                        if count > 0:
                            dot.edge(main_flow[i], main_flow[i+1], label=f'{count}', color='blue')
                
                # Add rework loops in red
                rework_activities = ['MOULDING', 'ASSEMBLY_1', 'ASSEMBLY_2', 'SORTING', 'PACKAGING']
                for activity in rework_activities:
                    if activity in self.models['dfg']['activities']:
                        rework_count = self.models['dfg']['dfg'].get((activity, activity), 0)
                        if rework_count > 0:
                            dot.edge(activity, activity, label=f'{rework_count}', color='red', style='dashed')
                
                dot.render(f'{output_dir}/process_flow', format='png', cleanup=True)
                print("✅ Saved Process Flow as process_flow.png")
                
        except Exception as e:
            print(f"⚠️ Direct visualization also failed: {e}")
            print("📊 Creating basic text-based visualization...")
            self.create_fallback_visualizations(output_dir)

    def create_fallback_visualizations(self, output_dir):
        """Create simple visualizations if PM4Py fails"""
        # Activity frequency chart
        activity_counts = {}
        if isinstance(self.event_log, pd.DataFrame):
            activity_counts = self.event_log['concept:name'].value_counts().to_dict()
        else:
            for event in self.event_log:
                activity = event['concept:name']
                activity_counts[activity] = activity_counts.get(activity, 0) + 1
        
        plt.figure(figsize=(12, 8))
        activities = list(activity_counts.keys())
        counts = list(activity_counts.values())
        
        plt.bar(activities, counts)
        plt.title('Activity Frequency in Manufacturing Process')
        plt.xlabel('Activities')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/activity_frequency.png", dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Saved activity frequency chart as activity_frequency.png")
        
        # Print DFG relations as text
        if 'dfg' in self.models:
            dfg = self.models['dfg']['dfg']
            print("\n🔗 Top Directly-Follows Relations:")
            for (from_act, to_act), count in sorted(dfg.items(), key=lambda x: x[1], reverse=True)[:15]:
                print(f"  {from_act} → {to_act}: {count} times")

    def run_complete_mining(self, run_id=0):
        """Run complete process mining pipeline"""
        print("=" * 70)
        print(f"🔨 COMPLETE PROCESS MINING - Run {run_id}")
        print("=" * 70)
        
        try:
            # 1. Prepare data
            self.prepare_event_log(run_id=run_id)
            
            # 2. Discover models
            dfg_model = self.discover_dfg()
            inductive_model = self.discover_inductive_miner()
            heuristics_model = self.discover_heuristics_miner()
            
            # 3. Analyze variants and metrics
            variants = self.analyze_process_variants()
            metrics = self.analyze_process_metrics()
            
            # 4. Compare methods
            comparison = self.compare_discovery_methods()
            
            # 5. Save results
            self.save_discovered_models()
            
            # 6. Save visualizations as images (guaranteed to work)
            self.save_all_visualizations()
            
            print("\n🎉 PROCESS MINING COMPLETE!")
            
            return {
                'models': self.models,
                'variants': variants,
                'metrics': metrics,
                'comparison': comparison
            }
            
        except Exception as e:
            print(f"❌ Process mining failed: {e}")
            import traceback
            print(f"🔍 Detailed traceback: {traceback.format_exc()}")
            return None

# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    # Load your data
    df = pd.read_csv("results/251020FIFO_l0.28_actuator_manufacturing_with_rework.csv")
    
    # Initialize miner
    miner = ProcessMiner(df)
    
    # Run complete mining on a single simulation run
    results = miner.run_complete_mining(run_id=0)
    
    if results:
        print("\n🎉 ALL DONE! Check these folders for results:")
        print("📁 'process_visualizations' - PNG images of process models")
        print("📁 'discovered_models' - CSV, PNML, and XES files")
    else:
        print("\n❌ Process mining failed. Check the error messages above.")

[OK] Graphviz reachable (created process_visualizations/_graphviz_ok.png)
🔨 COMPLETE PROCESS MINING - Run 0
⚠️ Event log conversion failed: list indices must be integers or slices, not str
📊 Using DataFrame directly: 1377 events, 1377 cases
🔄 Discovering Directly-Follows Graph...
✅ DFG discovered: 0 direct-follows relations
🔄 Discovering process with Inductive Miner...
✅ Inductive Miner model discovered
🔄 Discovering process with Heuristics Miner...
✅ Heuristics Miner model discovered
🔍 Analyzing process variants...
📋 Found 1 unique process variants

Top 10 most frequent variants:
 1. Count: 1377 - END
📊 Calculating process metrics...

📈 Process Metrics:
   Cases: 1377
   Average case duration: 0.00 minutes
   Throughput: 16.66 cases/hour
   Unique activities: 1

🏭 Most frequent activities:
   END: 1377 occurrences
⚖️ Comparing discovery methods...

🔬 Discovery Method Comparison:

DFG:
  num_activities: 0
  num_relations: 0
  start_activities: 1
  end_activities: 1
  description: Direc

c:\Users\990215322\AppData\Local\miniconda3\envs\mupro\Lib\site-packages\pm4py\utils.py:991: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
c:\Users\990215322\AppData\Local\miniconda3\envs\mupro\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 1377/1377 [00:00<00:00, 12689.18it/s]

✅ Saved event log to event_log.xes
💾 Saving visualizations as images...
⚠️ PM4Py visualization failed: string indices must be integers, not 'str'
🔄 Trying alternative approach...
🎨 Creating visualizations using graphviz directly...
✅ Saved DFG as dfg.png
✅ Saved Process Flow as process_flow.png

🎉 PROCESS MINING COMPLETE!

🎉 ALL DONE! Check these folders for results:
📁 'process_visualizations' - PNG images of process models
📁 'discovered_models' - CSV, PNML, and XES files


In [3]:
import graphviz
g = graphviz.Digraph()
g.edge('A','B')
g.render('test_graphviz', format='png')  # should write test_graphviz.png


CalledProcessError: Command '['dot.bat', '-Kdot', '-Tpng', '-O', 'test_graphviz']' returned non-zero exit status 3221225477. [stderr: b'Warning: Could not load "C:\\Users\\990215322\\AppData\\Local\\miniconda3\\envs\\mupro\\Library\\bin\\gvplugin_pango.dll" - It was found, so perhaps one of its dependents was not.  Try ldd.\r\nWarning: Could not load "C:\\Users\\990215322\\AppData\\Local\\miniconda3\\envs\\mupro\\Library\\bin\\gvplugin_pango.dll" - It was found, so perhaps one of its dependents was not.  Try ldd.\r\nWarning: Could not load "C:\\Users\\990215322\\AppData\\Local\\miniconda3\\envs\\mupro\\Library\\bin\\gvplugin_pango.dll" - It was found, so perhaps one of its dependents was not.  Try ldd.\r\n']